# Bowling Stats

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, glob, yaml, pickle
import datetime

from utils import get_player_profile

In [2]:
def add_overs(overs):
    ov_int = 0
    ov_frac = 0
    for ov in overs:
        ov_split = ov.split('.')
        ov_int  += int(ov_split[0])
        
        if len(ov_split)>1:
            ov_frac += int(ov.split('.')[1])
            
        if ov_frac >= 6:
            ov_int += ov_frac//6
            ov_frac = ov_frac %6

    if ov_frac==0:
        return str(ov_int)
    else:
        return str(ov_int)+'.'+str(ov_frac)

overs=['1.2', '3.5', '4', '2.5']
add_overs(overs)

'12'

In [3]:
def bowling_summary():
    df=pickle.load(open('./database/bowling_record_all_years.df', 'rb'))
    all_players = df['bowler'].unique()
    
    data=[]
    for player in all_players:
        
        dfp = df[ df['bowler'] == player ]
        Inns_ = dfp.shape[0]
        
        Ovs_  = add_overs(dfp['O'].values)
        Wkts_ = dfp['W'].sum()

        #HS_   = max(dfp.Runs) to be calculated for best bowling figures
        SR_   = 0.
        Ave_  = 0.
        Fours_= dfp['4s'].sum()
        Sixes_= dfp['6s'].sum()
        WDs_  = dfp['WD'].sum()
        NBs_  = dfp['NB'].sum()

        NoWs_= ((dfp['W']==0)).sum()
        ThreeWs_= ((dfp['W']>=3)).sum()
        FourWs_ = ((dfp['W']>=4)).sum()
        FiveWs_ = ((dfp['W']>=5)).sum()

        data.append([player, Inns_, Ovs_, Wkts_, SR_, Ave_, Fours_, Sixes_,
                     WDs_, NBs_, NoWs_, ThreeWs_, FourWs_, FiveWs_])
    
    df_p = pd.DataFrame(data, columns=['Player', 'Innings', 'Overs', 'Wickets', 
                                       'SR', 'Ave', 'Fours', 'Sixes', 'WDs', 'NBs',
                                       '0-Fers', '3-Fers', '4-Fers', '5-Fers']) 
    
    return df_p

df_summary= bowling_summary()
df_summary.head(2)

,Player,Innings,Overs,Wickets,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
0,P Kumar,119,420.4,90,0.0,0.0,338,104,112,1,57,3,0,0
1,Z Khan,99,366.4,103,0.0,0.0,312,64,74,2,35,10,1,0


## Top players in different categories

In [4]:
## Top players in terms of Runs, 50s, 100s, 4s, 6s and so ondef sorted_table(sort_by='Wickets', min_wkts=100):
def sorted_table(sort_by='Wickets', min_wkts=100, ascending=False):
    df_summary = bowling_summary()
    df_summary = df_summary[ (df_summary['Wickets']>=min_wkts) ]
    df_sorted  = df_summary.sort_values(by=[sort_by], ascending=ascending)
    return df_sorted

In [5]:
def get_top_players():
    sorting_list = ['Innings', 'Overs', 'Wickets', '3-Fers', 'Fours', 'Sixes', 'WDs', 'NBs']
    for sort_item in sorting_list:
        print ('Sorted by :',sort_item)

        if sort_item in ['Fours', 'Sixes', 'WDs', 'NBs']:
            df_sorted = sorted_table(sort_by=sort_item, ascending=True)
        else:
            df_sorted = sorted_table(sort_by=sort_item, ascending=False)

        display(df_sorted[:5])

get_top_players()

Sorted by : Innings


,Player,Innings,Overs,Wickets,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
27,PP Chawla,163,541.4,156,0.0,0.0,316,181,33,2,59,12,2,0
49,Harbhajan Singh,157,562.2,150,0.0,0.0,289,142,74,3,64,11,2,1
89,RA Jadeja,155,452.1,114,0.0,0.0,213,148,44,2,85,11,4,1
144,R Ashwin,151,538.2,138,0.0,0.0,226,135,95,2,56,7,1,0
84,A Mishra,150,526.5,160,0.0,0.0,222,172,51,21,54,16,4,1


Sorted by : Overs


,Player,Innings,Overs,Wickets,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
49,Harbhajan Singh,157,562.2,150,0.0,0.0,289,142,74,3,64,11,2,1
27,PP Chawla,163,541.4,156,0.0,0.0,316,181,33,2,59,12,2,0
144,R Ashwin,151,538.2,138,0.0,0.0,226,135,95,2,56,7,1,0
84,A Mishra,150,526.5,160,0.0,0.0,222,172,51,21,54,16,4,1
103,SL Malinga,122,471.1,170,0.0,0.0,314,86,129,18,31,19,7,1


Sorted by : Wickets


...

,Player,Innings,Overs,Wickets,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
1,Z Khan,99,366.4,103,0.0,0.0,312,64,74,2,35,10,1,0
45,A Nehra,88,318,106,0.0,0.0,279,76,57,9,26,14,1,0
251,JJ Bumrah,92,348.4,109,0.0,0.0,232,79,45,22,33,14,2,0
103,SL Malinga,122,471.1,170,0.0,0.0,314,86,129,18,31,19,7,1
211,B Kumar,121,449.3,136,0.0,0.0,336,88,89,9,43,12,3,1


Sorted by : WDs


,Player,Innings,Overs,Wickets,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
27,PP Chawla,163,541.4,156,0.0,0.0,316,181,33,2,59,12,2,0
231,SP Narine,119,464.1,127,0.0,0.0,234,107,37,2,43,12,7,1
89,RA Jadeja,155,452.1,114,0.0,0.0,213,148,44,2,85,11,4,1
251,JJ Bumrah,92,348.4,109,0.0,0.0,232,79,45,22,33,14,2,0
43,R Vinay Kumar,104,353.3,105,0.0,0.0,296,98,48,10,40,10,1,0


Sorted by : NBs


,Player,Innings,Overs,Wickets,SR,Ave,Fours,Sixes,WDs,NBs,0-Fers,3-Fers,4-Fers,5-Fers
1,Z Khan,99,366.4,103,0.0,0.0,312,64,74,2,35,10,1,0
27,PP Chawla,163,541.4,156,0.0,0.0,316,181,33,2,59,12,2,0
89,RA Jadeja,155,452.1,114,0.0,0.0,213,148,44,2,85,11,4,1
144,R Ashwin,151,538.2,138,0.0,0.0,226,135,95,2,56,7,1,0
231,SP Narine,119,464.1,127,0.0,0.0,234,107,37,2,43,12,7,1
